# High Injury Crash Network

## Setup

In [1]:
# import packages
import arcpy
from arcgis.features import FeatureLayer
import os
import pandas as pd
import numpy as np
from sqlalchemy.engine import URL, create_engine
from arcgis import GIS

# set overwrite to true
arcpy.env.overwriteOutput = True

# enterprise Geodatabase connection
sdeBase = "F:\GIS\DB_CONNECT\Vector.sde"

# set workspace - Need to modify this to be universal
arcpy.env.workspace = "F:\GIS\PROJECTS\Transportation\Vision Zero\CrashAnalysis\Crash Analysis.gdb"
workspace           = r"F:\GIS\PROJECTS\Transportation\Vision Zero\CrashAnalysis\Crash Analysis.gdb"
workspace_folder    = r"F:\GIS\PROJECTS\Transportation\Vision Zero\CrashAnalysis"

# in memory output file path
memory_workspace = "memory" + "\\"

# network dataset info
# nd_path =  r"F:\GIS\GIS_DATA\Transportation\Basemap Features\Roads\Streets Network Dataset\Streets_NEW_ND.gdb\Streets_SDC_ND\Streets_CA_NV_ND"
# nd_layer_name = "Streets_CA_NV"

# streetNetwork = os.path.join(nd_path,nd_layer_name)
#HAVE WE PUBLISHED THIS?
streets    = os.path.join(sdeBase, "sde.SDE.Transportation\sde.SDE.Streets")
urban      = os.path.join(sdeBase, "sde.SDE.Jurisdictions\sde.SDE.UrbanAreas")

streetNetwork = "Tahoe_OSM_Streets"

# crash data
service_url = 'https://maps.trpa.org/server/rest/services/Transportation/MapServer/0'

feature_layer = FeatureLayer(service_url)
query_result = feature_layer.query()
# Convert the query result to a list of dictionaries
feature_list = query_result.features

# Create a pandas DataFrame from the list of dictionaries
tahoeCrash_data = pd.DataFrame([feature.attributes for feature in feature_list])
# https://maps.trpa.org/server/rest/services/Transportation/MapServer/0
tahoeCrash   = os.path.join(sdeBase, "sde.SDE.Transportation\sde.SDE.Tahoe_Crash")

# crashTotal   = arcpy.MakeFeatureLayer_management(tahoeCrash, "Crashes")
# crashFatal   = arcpy.MakeFeatureLayer_management(tahoeCrash, "Fatal Crashes", "Crash_Severity = 'Fatal'")
# crashSevere  = arcpy.MakeFeatureLayer_management(tahoeCrash, "Severe Crashes", "Crash_Severity = 'Severe injury'")
# crashOther   = arcpy.MakeFeatureLayer_management(tahoeCrash, "Other Crashes", "Crash_Severity IN ('Complaint of pain', 'Other visible injury')")
# crashBikePed = arcpy.MakeFeatureLayer_management(tahoeCrash, "Bike/Ped Crashes", "Bicycle_Involved = 'Y' And Pedestrian_Involved = 'Y' And Crash_Severity NOT IN ('Fatal', 'Property damage only', 'Severe injury')")

# hex for crash data aggregations
hexFatal    = memory_workspace + "Hex_Fatal_Crash"
hexSevere   = memory_workspace + "Hex_Severe_Crash"
hexMinor    = memory_workspace + "Hex_Minor_Crash"
hexBikePed  = memory_workspace + "Hex_BikePed_Crash"
hexTotal    = memory_workspace + "Hex_Total_Crash"

Exception: Unable to complete operation.
(Error Code: 400)

### Functions

In [ ]:
# functions
# transfer attributes from spatial join feature class to tessellation
def fieldJoinCalc(updateFC, updateFieldsList, sourceFC, sourceFieldsList):
    from time import strftime  
    print ("Started data transfer: " + strftime("%Y-%m-%d %H:%M:%S"))
#     log.info("Started data transfer: " + strftime("%Y-%m-%d %H:%M:%S"))
    # Use list comprehension to build a dictionary from arcpy SearchCursor  
    valueDict = {r[0]:(r[1:]) for r in arcpy.da.SearchCursor(sourceFC, sourceFieldsList)}  
    with arcpy.da.UpdateCursor(updateFC, updateFieldsList) as updateRows:  
        for updateRow in updateRows:  
            # store the Join value of the row being updated in a keyValue variable  
            keyValue = updateRow[0]  
            # verify that the keyValue is in the Dictionary  
            if keyValue in valueDict:  
                # transfer the value stored under the keyValue from the dictionary to the updated field.  
                updateRow[1] = valueDict[keyValue][0]  
                updateRows.updateRow(updateRow)    
    del valueDict  
    print ("Finished data transfer: " + strftime("%Y-%m-%d %H:%M:%S"))


### Clean up Data

In [ ]:
#Deleting closed forest service road
object_id_to_delete = 9043  # Replace with the actual ObjectID you want to delete

# Use a delete cursor to find and delete the specified feature
with arcpy.da.UpdateCursor('Streets_Tahoe', ["OBJECTID"], "OBJECTID = {}".format(object_id_to_delete)) as cursor:
    for row in cursor:
        cursor.deleteRow()
        print("Feature with ObjectID {} deleted.".format(object_id_to_delete))


In [ ]:
where_clause = "Crash_Severity = 'Property damage only'"
arcpy.management.MakeFeatureLayer('Tahoe_Crash', 'tempLayer', where_clause)
arcpy.DeleteFeatures_management('tempLayer')

## Generate Tessellations

In [ ]:
tahoeHex = "TahoeTessellation_25000sqm"

arcpy.management.GenerateTessellation(
    Output_Feature_Class=tahoeHex,
    Extent='-13388021.2623077 4678916.64488463 -13342410.4544964 4769413.84969433 PROJCS["WGS_1984_Web_Mercator_Auxiliary_Sphere",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mercator_Auxiliary_Sphere"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],PARAMETER["Standard_Parallel_1",0.0],PARAMETER["Auxiliary_Sphere_Type",0.0],UNIT["Meter",1.0]]',
    Shape_Type="HEXAGON",
    Size="25000 SquareMeters",
    Spatial_Reference='PROJCS["NAD_1983_UTM_Zone_10N",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-123.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]];-5120900 -9998100 10000.0016462827;-100000 10000;-100000 10000;1.00000016391277E-03;0.001;0.001;IsHighPrecision',
    H3_Resolution=7
)

arcpy.management.SelectLayerByLocation(
    in_layer="TahoeTessellation_25000sqm",
    overlap_type="INTERSECT",
    select_features="TRPABoundary_Erase",
    search_distance=None,
    selection_type="NEW_SELECTION",
    invert_spatial_relationship="INVERT"
)

arcpy.management.DeleteRows(
    in_rows="TahoeTessellation_25000sqm"
)

In [ ]:
tahoeHex = "TahoeTessellation_25000sqm"
# Add fields to accept join values
inFeatures = tahoeHex
fieldPrecision = 9

fieldName1  = "SEVERE_CRASH_COUNT"
fieldAlias1 = "Number of Severe Crashes"

fieldName2  = "FATAL_CRASH_COUNT"
fieldAlias2 = "Number of Fatal Crashes"

fieldName3  = "OTHER_CRASH_COUNT"
fieldAlias3 = "Number of Minor Crashes"

fieldName4  = "BIKEPED_CRASH_COUNT"
fieldAlias4 = "Number of Bike/Ped Crashes"

fieldName5  = "CRASH_COUNT"
fieldAlias5 = "Total Crashes"

### Density Fields
# Run AddField 
arcpy.management.AddField(inFeatures, 
                          fieldName1, "DOUBLE", 
                          fieldPrecision,
                          field_alias=fieldAlias1, 
                          field_is_nullable="NULLABLE")
# Run AddField 
arcpy.management.AddField(inFeatures, 
                          fieldName2, "DOUBLE", 
                          fieldPrecision,
                          field_alias=fieldAlias2, 
                          field_is_nullable="NULLABLE")
# Run AddField 
arcpy.management.AddField(inFeatures, 
                          fieldName3, "DOUBLE", 
                          fieldPrecision,
                          field_alias=fieldAlias3, 
                          field_is_nullable="NULLABLE")
# Run AddField 
arcpy.management.AddField(inFeatures, 
                          fieldName4, "DOUBLE", 
                          fieldPrecision,
                          field_alias=fieldAlias4, 
                          field_is_nullable="NULLABLE")

# Run AddField 
arcpy.management.AddField(inFeatures, 
                          fieldName5, "DOUBLE", 
                          fieldPrecision,
                          field_alias=fieldAlias5, 
                          field_is_nullable="NULLABLE")
# Run AddField 
arcpy.management.AddField(inFeatures, 
                          "IS_URBAN", "TEXT", 
                          field_alias="Within Urban Area Boundary?", 
                          field_is_nullable="NULLABLE")

# Run AddField 
arcpy.management.AddField(inFeatures, 
                          "HAS_ROAD", "TEXT", 
                          field_alias="Road within Hex?", 
                          field_is_nullable="NULLABLE")


In [ ]:
### Crash Attribute Update -----------------------------------------------------------------------------------###
# Spatial Join
arcpy.SpatialJoin_analysis(tahoeHex, crashFatal, 
                           hexFatal, 
                           "JOIN_ONE_TO_ONE", "KEEP_ALL", "", 
                           "CONTAINS", "", "")

arcpy.SpatialJoin_analysis(tahoeHex, crashSevere, 
                           hexSevere, 
                           "JOIN_ONE_TO_ONE", "KEEP_ALL", "", 
                           "CONTAINS", "", "")

arcpy.SpatialJoin_analysis(tahoeHex, crashOther, 
                           hexMinor, 
                           "JOIN_ONE_TO_ONE", "KEEP_ALL", "", 
                           "CONTAINS", "", "")

arcpy.SpatialJoin_analysis(tahoeHex, crashBikePed, 
                           hexBikePed, 
                           "JOIN_ONE_TO_ONE", "KEEP_ALL", "", 
                           "CONTAINS", "", "")

arcpy.SpatialJoin_analysis(tahoeHex, crashTotal, 
                           hexTotal, 
                           "JOIN_ONE_TO_ONE", "KEEP_ALL", "", 
                           "CONTAINS", "", "")

# transfer attributes to hex Layer
fieldJoinCalc(tahoeHex, ['GRID_ID','FATAL_CRASH_COUNT'], 
              hexFatal,     ['GRID_ID','Join_Count'])
# transfer attributes to hex Layer
fieldJoinCalc(tahoeHex, ['GRID_ID','SEVERE_CRASH_COUNT'], 
              hexSevere,    ['GRID_ID','Join_Count'])
# transfer attributes to hex Layer
fieldJoinCalc(tahoeHex, ['GRID_ID','OTHER_CRASH_COUNT'], 
              hexMinor,     ['GRID_ID','Join_Count'])
# transfer attributes to hex Layer
fieldJoinCalc(tahoeHex, ['GRID_ID','BIKEPED_CRASH_COUNT'], 
              hexBikePed,   ['GRID_ID','Join_Count'])
# transfer attributes to hex Layer
fieldJoinCalc(tahoeHex, ['GRID_ID','CRASH_COUNT'], 
              hexTotal,   ['GRID_ID','Join_Count'])

### Urban Hex Attribute Update -----------------------------------------------------------------------------###

# Select parcels that intersect urban area boundary
urbanSelect = arcpy.SelectLayerByLocation_management(tahoeHex, 
                                                          'INTERSECT', 
                                                           urban, 
                                                           0, 
                                                          'NEW_SELECTION')
# Update field 1= yes 0 = no
with arcpy.da.UpdateCursor(urbanSelect, ['IS_URBAN']) as cursor:
    for row in cursor:
        row[0] = '1'
        cursor.updateRow(row) 
del cursor 

# switch the selection
urbanAreaSelect = arcpy.SelectLayerByAttribute_management(urbanSelect,'SWITCH_SELECTION')

# update other parcels
with arcpy.da.UpdateCursor(urbanAreaSelect, ['IS_URBAN']) as cursor:
    for row in cursor:
        row[0] = '0'
        cursor.updateRow(row)
del cursor

### Streets Hex Attribute Update -----------------------------------------------------------------------------###

# Select parcels that intersect urban area boundary
streetSelect = arcpy.SelectLayerByLocation_management(tahoeHex, 
                                                          'INTERSECT', 
                                                           streets, 
                                                           0, 
                                                          'NEW_SELECTION')
# Update field 1= yes 0 = no
with arcpy.da.UpdateCursor(streetSelect, ['HAS_ROAD']) as cursor:
    for row in cursor:
        row[0] = '1'
        cursor.updateRow(row) 
del cursor 

# switch the selection
streetNotSelect = arcpy.SelectLayerByAttribute_management(streetSelect,'SWITCH_SELECTION')

# update other parcels
with arcpy.da.UpdateCursor(streetNotSelect, ['HAS_ROAD']) as cursor:
    for row in cursor:
        row[0] = '0'
        cursor.updateRow(row)
del cursor

# clear selection
arcpy.SelectLayerByAttribute_management(tahoeHex, "CLEAR_SELECTION")

## Cluster Analysis

In [ ]:
tahoeHexRoad = arcpy.MakeFeatureLayer_management(tahoeHex01km, "Hex_Road_Layer", 
                        "HAS_ROAD = '1'")
arcpy.stats.HotSpots(
    Input_Feature_Class=tahoeHexRoad,
    Input_Field="FATAL_CRASH_COUNT",
    Output_Feature_Class=r"TahoeTessellation_FatalCrash_HotSpots",
    Conceptualization_of_Spatial_Relationships="FIXED_DISTANCE_BAND",
    Distance_Method="EUCLIDEAN_DISTANCE",
    Standardization="ROW",
    Distance_Band_or_Threshold_Distance=None,
    Self_Potential_Field=None,
    Weights_Matrix_File=None,
    Apply_False_Discovery_Rate__FDR__Correction="NO_FDR",
    number_of_neighbors=None
)

## Emerging Hot Spot Analysis - Space Time Cube

In [ ]:
# where_clause  = ""
crashLayer    = 'Crash_Layer'
crashes       = arcpy.management.MakeFeatureLayer("Tahoe_Crash", crashLayer)
cube = r"F:\GIS\PROJECTS\Transportation\Vision Zero\CrashAnalysis\CrashCube_Tahoe"
hotspotOut    = "EmergingHotSpot_Crash_Tahoe"

arcpy.stpm.CreateSpaceTimeCube(
    in_features=crashLayer,
    output_cube=cube,
    time_field="Date",
    template_cube=None,
    time_step_interval="6 Months",
    time_step_alignment="END_TIME",
    reference_time=None,
    distance_interval="500 Meters",
    summary_fields=None,
    aggregation_shape_type="HEXAGON_GRID",
    defined_polygon_locations=None,
    location_id=None
)

# run emerging hot spot
arcpy.stpm.EmergingHotSpotAnalysis(
    in_cube=cube,
    analysis_variable="COUNT",
    output_features= hotspotOut,
    neighborhood_distance=None,
    neighborhood_time_step=1,
    polygon_mask=None,
    conceptualization_of_spatial_relationships="FIXED_DISTANCE",
    number_of_neighbors=None,
    define_global_window="ENTIRE_CUBE"
)

In [ ]:
where_clause  = "State ='CA'"
crashLayer    = 'CA_Crash_Layer'
crashes       = arcpy.management.MakeFeatureLayer("Tahoe_Crash", crashLayer, where_clause)
cube = r"F:\GIS\PROJECTS\Transportation\Vision Zero\CrashAnalysis\CrashCube_CA"
# cube          = os.path.join(workspace,"CrashCube_CA")
hotspotOut    = "EmergingHotSpot_Crash_CA"

arcpy.stpm.CreateSpaceTimeCube(
    in_features=crashLayer,
    output_cube=cube,
    time_field="Date",
    template_cube=None,
    time_step_interval="6 Months",
    time_step_alignment="END_TIME",
    reference_time=None,
    distance_interval="500 Meters",
    summary_fields=None,
    aggregation_shape_type="HEXAGON_GRID",
    defined_polygon_locations=None,
    location_id=None
)

# run emerging hot spot
arcpy.stpm.EmergingHotSpotAnalysis(
    in_cube=cube,
    analysis_variable="COUNT",
    output_features= hotspotOut,
    neighborhood_distance=None,
    neighborhood_time_step=1,
    polygon_mask=None,
    conceptualization_of_spatial_relationships="FIXED_DISTANCE",
    number_of_neighbors=None,
    define_global_window="ENTIRE_CUBE"
)

In [ ]:
where_clause  = "State ='NV'"
crashLayer    = 'NV_Crash_Layer'
crashes       = arcpy.management.MakeFeatureLayer("Tahoe_Crash", crashLayer, where_clause)
cube = r"F:\GIS\PROJECTS\Transportation\Vision Zero\CrashAnalysis\CrashCube_NV"
# cube          = os.path.join(workspace,"CrashCube_CA")
hotspotOut    = "EmergingHotSpot_Crash_NV"

arcpy.stpm.CreateSpaceTimeCube(
    in_features=crashLayer,
    output_cube=cube,
    time_field="Date",
    template_cube=None,
    time_step_interval="6 Months",
    time_step_alignment="END_TIME",
    reference_time=None,
    distance_interval="500 Meters",
    summary_fields=None,
    aggregation_shape_type="HEXAGON_GRID",
    defined_polygon_locations=None,
    location_id=None
)

# run emerging hot spot
arcpy.stpm.EmergingHotSpotAnalysis(
    in_cube=cube,
    analysis_variable="COUNT",
    output_features= hotspotOut,
    neighborhood_distance=None,
    neighborhood_time_step=1,
    polygon_mask=None,
    conceptualization_of_spatial_relationships="FIXED_DISTANCE",
    number_of_neighbors=None,
    define_global_window="ENTIRE_CUBE"
)

## Safety Index

In [ ]:
tahoeHexUrban = arcpy.MakeFeatureLayer_management(tahoeHex01km, "Hex_Urban_Layer", 
                        "IS_URBAN = '1' Or HAS_ROAD = '1'")
 

with arcpy.EnvManager(scratchWorkspace=r"F:\GIS\PROJECTS\Transportation\Equity\Equity.gdb", workspace=r"F:\GIS\PROJECTS\Transportation\Equity\Equity.gdb"):
    arcpy.stats.CalculateCompositeIndex(
        in_table=tahoeHexUrban,
        in_variables="DISABILITY_DENSITY #;MEDICARE_ACCESS_DISTANCE false;FOOD_ACCESS_DISTANCE false;MAJORITY_SLOPE_CLASS #",
        append_to_input="NEW_FEATURES",
        out_table="TahoeTessellation_0_1sqkm_CalculateCompositeIndex",
        index_preset="MEAN_SCALED",
        preprocessing="MINMAX",
        pre_threshold_scaling="THRESHOLD_PERCENTILE",
        pre_custom_zscore=None,
        pre_min_max=None,
        pre_thresholds=None,
        index_method="MEAN",
        index_weights="DISABILITY_DENSITY 1;MEDICARE_ACCESS_DISTANCE 1;FOOD_ACCESS_DISTANCE 1;MAJORITY_SLOPE_CLASS 1",
        out_index_name="",
        out_index_reverse=None,
        post_min_max=None,
        post_reclass=None,
        post_num_classes=5,
        post_custom_classes=None
    )

## Crash Analysis by Segment

In [2]:
arcpy.AddField_management("Tahoe_Crash", 'Crash_Severity_Numeric', "LONG")
arcpy.AddField_management("Tahoe_Crash", 'Crash_Rate_Weighted', "DOUBLE")


<Result 'F:\\GIS\\PROJECTS\\Transportation\\Vision Zero\\CrashAnalysis\\Crash Analysis.gdb\\Tahoe_Crash'>

In [2]:
def crash_severity_numeric(field1_value, field2_value, field3_value):
    if field1_value == 'Fatal':
        return 5
    elif field1_value == 'Severe injury':
        return 3
    elif field2_value == 'Y':
        return 2
    elif field3_value == 'Y':
        return 2
    else:
        return 1  # Default value for other cases

# Use CalculateField_management to apply the function to the new field
expression = "crash_severity_numeric(!Crash_Severity!, !Bicycle_Involved!, !Pedestrian_Involved!)"  # Replace 'Field1' with your source field name
code_block = """def crash_severity_numeric(field1_value, field2_value, field3_value):
    if field1_value == 'Fatal':
        return 5
    elif field1_value == 'Severe injury':
        return 3
    elif field2_value == 'Y':
        return 2
    elif field3_value == 'Y':
        return 2
    else:
        return 1 """  # Define the function here

arcpy.CalculateField_management("Tahoe_Crash", "Crash_Severity_Numeric", expression, "PYTHON3", code_block)
print("Calculation complete.")

Calculation complete.


In [3]:
arcpy.AddField_management("Tahoe_Crash", 'Bicycle_Involved_Numeric', "LONG")
expression = "crash_severity_numeric(!Bicycle_Involved!)"  # Replace 'Field1' with your source field name
code_block = """def crash_severity_numeric(field1_value):
    if field1_value == 'Y':
        return 1
    else:
        return 0 """  # Define the function here

arcpy.CalculateField_management("Tahoe_Crash", "Bicycle_Involved_Numeric", expression, "PYTHON3", code_block)
print("Calculation complete.")

Calculation complete.


In [4]:
arcpy.AddField_management("Tahoe_Crash", 'Pedestrian_Involved_Numeric', "LONG")
expression = "crash_severity_numeric(!Pedestrian_Involved!)"  # Replace 'Field1' with your source field name
code_block = """def crash_severity_numeric(field1_value):
    if field1_value == 'Y':
        return 1
    else:
        return 0 """  # Define the function here

arcpy.CalculateField_management("Tahoe_Crash", "Pedestrian_Involved_Numeric", expression, "PYTHON3", code_block)
print("Calculation complete.")

Calculation complete.


In [5]:
arcpy.edit.Snap(
    in_features="Tahoe_Crash",
    snap_environment="Streets_Tahoe EDGE '0.25 Miles'"
)

<Result 'F:\\GIS\\PROJECTS\\Transportation\\Vision Zero\\CrashAnalysis\\Crash Analysis.gdb\\Tahoe_Crash'>

In [ ]:
target_feature_class = "Streets_Tahoe"
join_feature_class = r"F:\GIS\PROJECTS\Transportation\Vision Zero\CrashAnalysis\Crash Analysis.gdb\Tahoe_Crash"
output_feature_class = r"F:\GIS\PROJECTS\Transportation\Vision Zero\CrashAnalysis\Crash Analysis.gdb\Streets_Tahoe_Crashes"

# List of input and join field names
input_field_names = ["NAME", "TYPE", "CLASS_RTE", "SPEED_MPH", "Shape_Length", "UniqueID", "CrashRate", "Miles", "FatalityRate"]
join_field_names = ["Num_Killed", "Num_Injured", "Num_Ped_Killed", "Num_Ped_Injured", 
    "Num_Bicyclist_Killed", "Num_Bicyclist_Injured", 
    "Crash_Severity_Numeric", "Crash_Rate_Weighted",
    "Bicycle_Involved_Numeric", "Pedestrian_Involved_Numeric"]

# List of fields to be joined using SUM
fields_to_sum = ["Num_Killed", "Num_Injured", "Num_Ped_Killed", "Num_Ped_Injured", 
                 "Num_Bicyclist_Killed", "Num_Bicyclist_Injured", "Crash_Severity_Numeric",
                "Bicycle_Involved_Numeric", "Pedestrian_Involved_Numeric"]

# Create a field map object
field_mappings = arcpy.FieldMappings()

# Get the field info for the target feature class
target_field_info = arcpy.ListFields(target_feature_class)

# Add fields from the target feature class to the field mappings
for field in target_field_info:
    if field.name in input_field_names:
        input_field_map = arcpy.FieldMap()
        input_field_map.addInputField(target_feature_class, field.name)
        
        # Set the merge rule to SUM for selected fields
        if field.name in fields_to_sum:
            input_field_map.mergeRule = "SUM"
        
        field_mappings.addFieldMap(input_field_map)

# Get the field info for the join feature class
join_field_info = arcpy.ListFields(join_feature_class)

# Add fields from the join feature class to the field mappings
for field in join_field_info:
    if field.name in join_field_names:
        join_field_map = arcpy.FieldMap()
        join_field_map.addInputField(join_feature_class, field.name)
        
        # Set the merge rule to SUM for selected fields
        if field.name in fields_to_sum:
            join_field_map.mergeRule = "SUM"
        
        field_mappings.addFieldMap(join_field_map)

# Add a field map for the number_of_records field (count of a specific field)
num_records_field_map = arcpy.FieldMap()
num_records_field_map.addInputField(join_feature_class, "County")
num_records_output_field = num_records_field_map.outputField
num_records_output_field.name = "Number_Of_Crashes"  # Change to your desired field name
num_records_output_field.aliasName = "Number Of Crashes"
num_records_output_field.type = "LONG"  # Change to appropriate data type
num_records_field_map.outputField = num_records_output_field
num_records_field_map.mergeRule = "COUNT"
field_mappings.addFieldMap(num_records_field_map)

# Perform the spatial join using the specified field mappings
arcpy.analysis.SpatialJoin(target_feature_class, 
    join_feature_class, 
    output_feature_class,
    join_operation="JOIN_ONE_TO_ONE",
    join_type="KEEP_ALL",
    field_mapping = field_mappings,
    match_option="INTERSECT",
    search_radius=None,
    distance_field_name="")

In [ ]:
arcpy.management.CalculateField(
    in_table="Streets_Tahoe_Crashes",
    field="Miles",
    expression="!Shape_Length! *  0.000621371",
    expression_type="PYTHON3",
    code_block="",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

arcpy.management.CalculateField(
    in_table="Streets_Tahoe_Crashes",
    field="Crash_Severity_Numeric",
    expression="0 if !Crash_Severity_Numeric! == None else !Crash_Severity_Numeric!",
    expression_type="PYTHON3",
    code_block="",
    field_type="FLOAT",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

arcpy.management.CalculateField(
    in_table="Streets_Tahoe_Crashes",
    field="Pedestrian_Involved_Numeric",
    expression="0 if !Pedestrian_Involved_Numeric! == None else !Pedestrian_Involved_Numeric!",
    expression_type="PYTHON3",
    code_block="",
    field_type="FLOAT",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

arcpy.management.CalculateField(
    in_table="Streets_Tahoe_Crashes",
    field="Bicycle_Involved_Numeric",
    expression="0 if !Bicycle_Involved_Numeric! == None else !Bicycle_Involved_Numeric!",
    expression_type="PYTHON3",
    code_block="",
    field_type="FLOAT",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

arcpy.management.CalculateField(
    in_table="Streets_Tahoe_Crashes",
    field="Number_Of_Crashes",
    expression="0 if !Number_Of_Crashes! == None else !Number_Of_Crashes!",
    expression_type="PYTHON3",
    code_block="",
    field_type="FLOAT",
    enforce_domains="NO_ENFORCE_DOMAINS"
)


arcpy.management.CalculateField(
    in_table="Streets_Tahoe_Crashes",
    field="Crash_Rate_Weighted",
    expression="!Crash_Severity_Numeric! / (!Miles!*8)",
    expression_type="PYTHON3",
    code_block="",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

arcpy.management.CalculateField(
    in_table="Streets_Tahoe_Crashes",
    field="Crash_Rate_Ped",
    expression="!Pedestrian_Involved_Numeric! / (!Miles!*8)",
    expression_type="PYTHON3",
    code_block="",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

arcpy.management.CalculateField(
    in_table="Streets_Tahoe_Crashes",
    field="Crash_Rate_Bike",
    expression="!Bicycle_Involved_Numeric! / (!Miles!*8)",
    expression_type="PYTHON3",
    code_block="",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

arcpy.management.CalculateField(
    in_table="Streets_Tahoe_Crashes",
    field="Crash_Rate_Total",
    expression="!Number_Of_Crashes! / (!Miles!*8)",
    expression_type="PYTHON3",
    code_block="",
    enforce_domains="NO_ENFORCE_DOMAINS"
)


#### Add Fields

In [ ]:
arcpy.AddField_management("Streets_Tahoe_Crashes", 'ped_HIN_0', "SHORT")
arcpy.AddField_management("Streets_Tahoe_Crashes", 'ped_HIN_05', "SHORT")
arcpy.AddField_management("Streets_Tahoe_Crashes", 'ped_HIN_tenth', "SHORT")
arcpy.AddField_management("Streets_Tahoe_Crashes", 'bike_HIN_0', "SHORT")
arcpy.AddField_management("Streets_Tahoe_Crashes", 'bike_HIN_tenth', "SHORT")
arcpy.AddField_management("Streets_Tahoe_Crashes", 'bike_HIN_05', "SHORT")
arcpy.AddField_management("Streets_Tahoe_Crashes", 'car_HIN_0', "SHORT")
arcpy.AddField_management("Streets_Tahoe_Crashes", 'car_HIN_tenth', "SHORT")
arcpy.AddField_management("Streets_Tahoe_Crashes", 'car_HIN_05', "SHORT")

### Create totals and rates of victims by mile/mode

In [ ]:
#Convert feature class to dataframe

crash_df = pd.DataFrame.spatial.from_featureclass("Streets_Tahoe_Crashes")

Summary_Fields = ['Num_Killed', 'Num_Injured', 'Num_Ped_Killed','Num_Ped_Injured', 'Num_Bicyclist_Killed', 'Num_Bicyclist_Injured']

crash_df[Summary_Fields]= crash_df[Summary_Fields].fillna(0)
crash_df['Total_Victims'] = crash_df['Num_Killed'] + crash_df['Num_Injured']
crash_df['Total_Ped']= crash_df['Num_Ped_Killed'] +crash_df['Num_Ped_Injured']
crash_df['Total_Bicyclist']= crash_df['Num_Bicyclist_Killed'] + crash_df['Num_Bicyclist_Injured']
crash_df['Total_Car'] = crash_df['Total_Victims'] - (crash_df['Total_Ped'] + crash_df['Total_Bicyclist'])
crash_df['Victims_Per_Mile'] = crash_df['Total_Victims']/crash_df['Miles']

crash_df['Car_Victims_Per_Mile']  =crash_df['Total_Car']/crash_df['Miles']
crash_df['Bike_Victims_Per_Mile'] =crash_df['Total_Bicyclist']/crash_df['Miles']
crash_df['Ped_Victims_Per_Mile']  =crash_df['Total_Ped']/crash_df['Miles']


### Create HIN by Mode
#### Selected segments sorted by rate of victims per mile until 65% of victims are accounted for

In [ ]:
def assign_value(row, threshold):
    if row['Miles'] > threshold:
        return 'above_threshold'
    else:
        return 'below_threshold'

def identify_HIN_segments(df, segment_threshold, victim_field, unique_id_field, rank_field):
    df['threshold_status']=df.apply(assign_value, args=(segment_threshold,), axis = 1)
    df_sorted = df.sort_values(by=rank_field, ignore_index = True, ascending = False)
    total_victims = df_sorted[victim_field].sum()
    df_sorted = df_sorted.loc[df_sorted['threshold_status']=='above_threshold']
    df_sorted['Cumulative_Vics'] = df_sorted[victim_field].cumsum()
    df_HIN = df_sorted[df_sorted['Cumulative_Vics']<= (.65 * total_victims)]
    HIN_IDs = df_HIN[unique_id_field]
    return HIN_IDs

# run for different segment lenghth thresholds and for mode types
Ped_HIN_0_ID      = identify_HIN_segments(crash_df, 0, 'Pedestrian_Involved_Numeric', 'UniqueID','Crash_Rate_Ped')
Ped_HIN_05_ID     = identify_HIN_segments(crash_df, 0.05, 'Pedestrian_Involved_Numeric', 'UniqueID','Crash_Rate_Ped')
Ped_HIN_tenth_ID  = identify_HIN_segments(crash_df, 0.1, 'Pedestrian_Involved_Numeric', 'UniqueID','Crash_Rate_Ped')
Bike_HIN_0_ID     = identify_HIN_segments(crash_df, 0, 'Bicycle_Involved_Numeric', 'UniqueID','Crash_Rate_Bike')
Bike_HIN_05_ID    = identify_HIN_segments(crash_df, 0.05, 'Bicycle_Involved_Numeric', 'UniqueID','Crash_Rate_Bike')
Bike_HIN_tenth_ID = identify_HIN_segments(crash_df, 0.1, 'Bicycle_Involved_Numeric', 'UniqueID','Crash_Rate_Bike')
Car_HIN_0_ID      = identify_HIN_segments(crash_df, 0, 'Total_Car', 'UniqueID','Car_Victims_Per_Mile')
Car_HIN_tenth_ID  = identify_HIN_segments(crash_df, 0.1, 'Total_Car', 'UniqueID','Car_Victims_Per_Mile')
Car_HIN_05_ID     = identify_HIN_segments(crash_df, 0.05, 'Total_Car', 'UniqueID','Car_Victims_Per_Mile')

In [ ]:
# 
def addHIN(fc, idList, field_name):
    # Update the new field based on presence in the list of Object IDs
    with arcpy.da.UpdateCursor(fc, ['UniqueID', field_name]) as cursor:
        for row in cursor:
            value = row[0]
            if value in idList.values:
                row[1] = 1
            else:
                row[1] = 0   
            cursor.updateRow(row)

addHIN("Streets_Tahoe_Crashes", Ped_HIN_0_ID, "ped_HIN_0")

addHIN("Streets_Tahoe_Crashes", Ped_HIN_05_ID, "ped_HIN_05")

addHIN("Streets_Tahoe_Crashes", Ped_HIN_tenth_ID, "ped_HIN_tenth")

addHIN("Streets_Tahoe_Crashes", Bike_HIN_0_ID, "bike_HIN_0")

addHIN("Streets_Tahoe_Crashes", Bike_HIN_05_ID, "bike_HIN_05")

addHIN("Streets_Tahoe_Crashes", Bike_HIN_tenth_ID, "bike_HIN_tenth")

addHIN("Streets_Tahoe_Crashes", Car_HIN_0_ID, "car_HIN_0")

addHIN("Streets_Tahoe_Crashes", Car_HIN_05_ID, "car_HIN_05")

addHIN("Streets_Tahoe_Crashes", Car_HIN_tenth_ID, "car_HIN_tenth")


### Hot Spot Analysis of segments by Weighted Crash Severity

In [ ]:
arcpy.stats.HotSpots(
    Input_Feature_Class="Streets_Tahoe_Crashes",
    Input_Field="Crash_Rate_Weighted",
    Output_Feature_Class=r"F:\GIS\PROJECTS\Transportation\Vision Zero\CrashAnalysis\Crash Analysis.gdb\Streets_Tahoe_Crashes_HotSpots",
    Conceptualization_of_Spatial_Relationships="FIXED_DISTANCE_BAND",
    Distance_Method="EUCLIDEAN_DISTANCE",
    Standardization="ROW",
    Distance_Band_or_Threshold_Distance=100,
    Self_Potential_Field=None,
    Weights_Matrix_File=None,
    Apply_False_Discovery_Rate__FDR__Correction="NO_FDR",
    number_of_neighbors=None
)